# Traffic Detection using Image Classification and Object Detection
 Two different model architectures would be used to train the custom model. For image classification and object detection, respectively, 
 the **EfficientNet** and **YOLO** models were chosen for their reliability and fast inference times, effective for real-time processing.

### Dataset
- COCO 2017 https://cocodataset.org/ 
- Traffic-Netv1 https://github.com/OlafenwaMoses/Traffic-Net/releases

Importing COCO 

In [12]:
import fiftyone as fo
import fiftyone.zoo as foz

In [40]:
# Define the classes and maximum samples per class
classes = ["person", "bicycle", "motorcycle", "bus", "truck", "car"]

# Load COCO dataset with annotations filtered for classes
detection_dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=["train","validation"],
    label_types=["detections"],
    classes=classes,
    include_id=False,
    include_license=False,
    only_matching=True,
    seed=12,
    max_samples=5000, 
    progress=True
)


 100% |██████|    1.9Gb/1.9Gb [1.6m elapsed, 0s remaining, 26.3Mb/s]        
Extracting annotations to '/Users/bualoydgreat/fiftyone/coco-2017/raw/instances_train2017.json'
 100% |████████████████| 5000/5000 [18.0m elapsed, 0s remaining, 2.9 images/s]      
Writing annotations for 5000 downloaded samples to '/Users/bualoydgreat/fiftyone/coco-2017/train/labels.json'
Found annotations at '/Users/bualoydgreat/fiftyone/coco-2017/raw/instances_val2017.json'
Only found 2968 (<5000) samples matching your requirements
 100% |████████████████| 2968/2968 [11.0m elapsed, 0s remaining, 3.0 images/s]      
Writing annotations for 2968 downloaded samples to '/Users/bualoydgreat/fiftyone/coco-2017/validation/labels.json'
Dataset info written to '/Users/bualoydgreat/fiftyone/coco-2017/info.json'
Loading 'coco-2017' split 'train'
 100% |███████████████| 5000/5000 [9.2s elapsed, 0s remaining, 546.0 samples/s]       
Loading 'coco-2017' split 'validation'
 100% |███████████████| 2968/2968 [5.6s elapsed, 

In [ ]:
session = fo.launch_app(detection_dataset)

In [43]:
# Export the filtered dataset to COCO format
export_dir = "/Users/bualoydgreat/PERSONAL-TEMPS/optimized-traffic-light-computer-vision/dataset/yolo"

# The splits to export
splits = ["train", "validation"]

# Export the splits
for split in splits:
    split_view = detection_dataset.match_tags(split)
    split_view.export(
        export_dir=export_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        label_field="detections",
        split=split,
        classes=classes,
    )

 100% |███████████████| 5000/5000 [8.0s elapsed, 0s remaining, 687.3 samples/s]       
Directory '/Users/bualoydgreat/PERSONAL-TEMPS/optimized-traffic-light-computer-vision/dataset/yolo' already exists; export will be merged with existing files
 100% |███████████████| 2968/2968 [4.4s elapsed, 0s remaining, 716.8 samples/s]      


Importing Custom Dataset

In [13]:
# The directory containing the dataset to import
dataset_dir = "/Users/bualoydgreat/PERSONAL-TEMPS/optimized-traffic-light-computer-vision/dataset/coco"

# The type of the dataset being imported
dataset_type = fo.types.COCODetectionDataset # for example

dataset = fo.Dataset.from_dir(
    dataset_dir=dataset_dir,
    dataset_type=dataset_type,
    name="screenshot_100",
)

session = fo.launch_app(dataset)

 100% |█████████████████| 100/100 [1.8s elapsed, 0s remaining, 56.5 samples/s]      


In [16]:
import fiftyone.utils.random as four

four.random_split(dataset, {"train": 0.85, "val": 0.15})
train_view = dataset.match_tags("train")
val_view = dataset.match_tags("test")


 100% |███████████████████| 99/99 [1.0s elapsed, 0s remaining, 94.3 samples/s]          
 100% |█████████████████████| 0/0 [1.4ms elapsed, ? remaining, ? samples/s] 


In [14]:
# Export the filtered dataset to COCO format
export_dir = "/Users/bualoydgreat/PERSONAL-TEMPS/optimized-traffic-light-computer-vision/dataset/custom-100"


split_view = detection_dataset.match_tags(split)
split_view.export(
    export_dir=export_dir,
    dataset_type=fo.types.YOLOv5Dataset,
    label_field="detections",
    split="train",
    classes=classes,
)

NameError: name 'splits' is not defined

Importing Traffic-Net v1

In [47]:
!curl -L https://github.com/OlafenwaMoses/Traffic-Net/releases/download/1.0/trafficnet_dataset_v1.zip > trafficnet.zip
!unzip trafficnet.zip
#!sudo rm trafficnet.zip
#!sudo mv trafficnet_dataset_v1 /dataset/trafficnet

## Object Detection
Using Ultralytics YOLOv8 to train the model.

### Training

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Train the model using M1/M2
results = model.train(data='dataset.yaml', epochs=100, imgsz=640, device='mps')

We might not need to train the Object Detection model since YOLOv8 is already trained using the COCO dataset. We just need the right classes to use.

In [2]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")
model.names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

The classes we need from the COCO dataset are the following:
```txt
0: 'person'
1: 'bicycle'
2: 'car'
3: 'motorcycle'
5: 'bus'
7: 'truck',
```

In [ ]:

model.predict(source="0", show=True, stream=True, classes=[0])
for i, (result) in enumerate(results):
    print('Do something with classes')

## Image Classification
Using Traffic-Net V1 dataset to train the an image classification model.